# I. Métodos Ensamblados - Bagging

¿Por qué estamos aprendiendo sobre Ensamblados?

- Método muy popular para mejorar el rendimiento predictivo de los modelos de aprendizaje automático
- Proporciona una base para comprender modelos más sofisticados

## Objetivos de la lección

Los estudiantes podrán:

- Definir el ensamblaje y sus requisitos
- Identificar los dos métodos básicos de ensamblaje
- Decidir si el ensamblaje manual es un enfoque útil para un problema dado
- Explicar el bagging y cómo se puede aplicar a los árboles de decisión
- Explicar cómo se calculan el error **"out-of-bag"** y las importancias de las  características de un bagged tree.
- Explicar la diferencia entre bagged trees and Random Forests
- Construir y ajustar un modelo de Random Forest en scikit-learn
- Decidir si un Árbol de Decisión o un Random Forest es mejor modelo para un problema dado.

# 1. Introducción

El aprendizaje ensamblado es un tema ampliamente estudiado en la comunidad de Machine Learning. La idea principal detrás
de la metodología de ensamblaje es combinar varios clasificadores base para tener un clasificador que supera a cada uno de ellos.

Cuando tratamos de predecir la variable Target utilizando cualquier técnica de Machine Learning, las causas principales de la diferencia en los valores reales y de predicción son el ruido, la varianza y el sesgo. El aprendizaje ensamblado ayuda a reducir estos factores (excepto el ruido, que es un error irreductible).

El principio central en el aprendizaje ensamblado es **inducir perturbaciones aleatorias en el procedimiento de aprendizaje para producir varios clasificadores base a partir de un solo conjunto de entrenamiento, y luego combinar los resultados de los clasificadores base para poder realizar la predicción final**. Para inducir las permutaciones aleatorias y, por lo tanto, crear los diferentes clasificadores base, se han propuesto varios métodos, en particular:
* Bagging
* Pasting
* Random Forests 
* Random Patches  

Finalmente, después de que los clasificadores base hayan sido entrenados, generalmente se combinan usando cualquiera de los siguientes métodos:
* Mayoría de votos
* Votación ponderada  
* Stacking


Hay tres razones principales con respecto a por qué los métodos ensamblados funcionan mejor que los modelos individuales: estadística, computacional y representacional. 

Primero, desde un punto de vista estadístico, cuando el conjunto de aprendizaje es demasiado pequeño, un algoritmo puede encontrar varios modelos buenos dentro del espacio de búsqueda, que surgen para el mismo rendimiento en el conjunto de entrenamiento. Sin embargo, sin un conjunto de validación, existe el riesgo de elegir el modelo incorrecto. 

La segunda razón es computacional; en general, los algoritmos se basan en una optimización de búsqueda local y pueden quedar atrapados en un optima local. Entonces, un ensamblado puede resolver esto enfocando diferentes algoritmos a diferentes espacios en el conjunto de entrenamiento. 

La última razón es representacional. En la mayoría de los casos, para un conjunto de aprendizaje de tamaño finito, la verdadera función $ f $ no puede representarse con ninguno de los modelos candidatos. Al combinar varios modelos en un conjunto, es posible obtener un modelo con una mayor cobertura en el espacio de funciones representables.

![](ch9_fig1.png)

## Ejemplo

Imaginemos que en lugar de construir un único modelo para resolver un problema de clasificación binario, creo **cinco modelos independientes**, y cada modelo predice de forma correcta aproximadamente el 70% de las veces. Si combina estos modelos en un "ensamblado (conjunto)" y utiliza su voto mayoritario como una predicción, ¿con qué frecuencia el ensamblado sería correcto?

In [ ]:
import numpy as np

# establecer una semilla para la reproducibilidad
np.random.seed(1234)

# generar 1000 números random (entre 0 y 1) para cada modelo, representando 1000 observaciones
mod1 = np.random.rand(1000)
mod2 = np.random.rand(1000)
mod3 = np.random.rand(1000)
mod4 = np.random.rand(1000)
mod5 = np.random.rand(1000)

# each model independently predicts 1 (the "correct response") if random number was at least 0.3
preds1 = np.where(mod1 > 0.3, 1, 0)
preds2 = np.where(mod2 > 0.3, 1, 0)
preds3 = np.where(mod3 > 0.3, 1, 0)
preds4 = np.where(mod4 > 0.3, 1, 0)
preds5 = np.where(mod5 > 0.3, 1, 0)

# print the first 20 predictions from each model
print(preds1[:20])
print(preds2[:20])
print(preds3[:20])
print(preds4[:20])
print(preds5[:20])

In [ ]:
# average the predictions and then round to 0 or 1
ensemble_preds = np.round((preds1 + preds2 + preds3 + preds4 + preds5)/5.0).astype(int)

# print the ensemble's first 20 predictions
print(ensemble_preds[:20])

In [ ]:
# ¿Qué tan preciso era cada modelo individual?
print(preds1.mean())
print(preds2.mean())
print(preds3.mean())
print(preds4.mean())
print(preds5.mean())

In [ ]:
# ¿Cuál fue la precisión del ensamblado?
print(ensemble_preds.mean())

** Nota: ** A medida que agrega más modelos al proceso de votación, la probabilidad de error disminuye, lo que se conoce como [Condorcet's Jury Theorem](http://en.wikipedia.org/wiki/Condorcet%27s_jury_theorem).

## ¿Qué es Ensamblado?

** El aprendizaje ensamblado (o "conjunto") ** es el proceso de combinar varios modelos predictivos para producir un modelo combinado que es más preciso que cualquier modelo individual.

- ** Regresión: ** tomar el promedio de las predicciones
- ** Clasificación: ** vote y use la predicción más común, o tome el promedio de las probabilidades pronosticadas

Para que el conjunto funcione bien, los modelos deben tener las siguientes características:

- ** Exacto: ** superan al modelo nulo
- ** Independiente: ** sus predicciones se generan utilizando diferentes procesos

** La gran idea: ** Si tiene una colección de modelos individualmente imperfectos (e independientes), los errores "únicos" hechos por cada modelo probablemente no serán hechos por el resto de los modelos, y por lo tanto el los errores se descartarán al promediar los modelos.

Hay dos métodos básicos ** para el ensamblado: **

- Ensamble manual de sus modelos individuales
- Use un modelo que ensamble por usted

### Rendimiento teórico de un ensamblado
  Si suponemos que cada uno de los $ T $ clasificadores base tiene una probabilidad $ \rho $ de predecir correctamente, la probabilidad de que un ensamblado tome la decisión correcta, asumiendo la independencia de los clasificadores, indicada por $ P_c $, se puede calcular utilizando la distribución binomial:

$$P_c = \sum_{j>T/2}^{T} {{T}\choose{j}} \rho^j(1-\rho)^{T-j}.$$

  Además, como se muestra, si $ T \ge3 $ entonces:

$$
  \lim_{T \to  \infty} P_c= \begin{cases}
            1  &\mbox{if } \rho>0.5 \\ 
            0  &\mbox{if } \rho<0.5 \\ 
            0.5  &\mbox{if } \rho=0.5 ,
            \end{cases}
$$
	llevando a la conclusión de que
$$
  \rho \ge 0.5 \quad \text{and} \quad T\ge3 \quad \Rightarrow \quad P_c\ge \rho.
$$

# 2. Ensamblaje Manual

¿Qué hace un buen modelo ensamblado manual?

- Diferentes tipos de ** modelos **
- Diferentes combinaciones de ** funciones **
- Diferentes ** parámetros de ajuste **

![Machine learning flowchart](https://raw.githubusercontent.com/justmarkham/DAT8/master/notebooks/images/crowdflower_ensembling.jpg)

*Machine learning flowchart created by the [winner](https://github.com/ChenglongChen/Kaggle_CrowdFlower) of Kaggle's [CrowdFlower competition](https://www.kaggle.com/c/crowdflower-search-relevance)*

In [ ]:
# Leer el dataset de entrenamiento
import pandas as pd
url = 'data/vehicles_train.csv'
train = pd.read_csv(url)
# Transformar la variable "vtype"
train['vtype'] = train.vtype.map({'car':0, 'truck':1})
# Leer el dataset de test
url = 'data/vehicles_test.csv'
test = pd.read_csv(url)
# Transformar la variable "vtype"
test['vtype'] = test.vtype.map({'car':0, 'truck':1})

In [ ]:
train.head()

### Entrenar diferentes modelos

En un diccionario, instanciar 4 técnicas de Aprendizaje Supervisado vistos hasta ahora, y que nos podrían ayudar a resolver este problema de regresión:
- Regresión Lineal
- Árbol de Regresión
- Naive Bayes
- KNN

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor

models = {'lr': LinearRegression(),
          'dt': DecisionTreeRegressor(),
          'nb': GaussianNB(),
          'nn': KNeighborsRegressor()}

Definir los predictores

In [ ]:
predictores = ['year', 'miles', 'doors', 'vtype']

Definir los datasets de entrenamiento y prueba, y entrenar con cada técnica

In [ ]:
# Entrenar todos los modelos
X_train = train.loc[:, predictores]
X_test = test.loc[:, predictores]
y_train = train.price
y_test = test.price

for model in models.keys():
    models[model].fit(X_train, y_train)

Ahora debemos predecir con los datos de Prueba usando cada uno de los 4 modelos entrenados. Para ello definamos un DataFrame con los mismos índices que el DataFrame de Prueba y las columnas con los nombres de las técnicas usadas.

In [ ]:
y_pred = pd.DataFrame(index=test.index, columns=models.keys())

Usando cada técnica, hay que predecir el precio:

In [ ]:
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

Ahora evaluaremos el poder predictivo de cada técnica. Para ello importemos la métrica **mean_squared_error**

In [ ]:
from sklearn.metrics import mean_squared_error

Usando cada técnica, hay que evaluar el error obtenido por sus predicciones:

In [ ]:
for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

### Evaluar el error de la media de las predicciones

In [ ]:
y_pred.mean(axis=1)

In [ ]:
np.sqrt(mean_squared_error(y_pred.mean(axis=1), y_test))

¿Podemos utilizar la media ponderada?

In [ ]:
weights = (2, 4, 1, 3)

In [ ]:
np.sqrt(mean_squared_error(np.average(y_pred, weights=weights, axis=1), y_test))

Podemos hacer combinaciones de pesos

In [ ]:
import itertools

In [ ]:
dErrors = {}
lWeights = list(itertools.product(range(1, 5), repeat=4))
for i, weights in enumerate(lWeights):
    dErrors[i] = np.sqrt(mean_squared_error(np.average(y_pred, weights=weights, axis=1), y_test))

In [ ]:
minWeight = lWeights[min(dErrors, key=dErrors.get)]
minWeight

In [ ]:
np.sqrt(mean_squared_error(np.average(y_pred, weights=minWeight, axis=1), y_test))

## Comparación del ensamblaje manual con el enfoque de un solo modelo


**Ventajas del ensamblaje manual:**

- Incrementa la precisión predictiva
- Fácil de comenzar

** Desventajas del ensamblaje manual:**

- Disminuye la interpretabilidad
- Toma más tiempo para entrenar
- Toma más tiempo para predecir
- Más complejo para automatizar y mantener

# 3. Bagging

La principal debilidad de los ** árboles de decisión ** es que no tienden a tener la mejor precisión predictiva. Esto se debe en parte a la **alta varianza**, lo que significa que diferentes divisiones en los datos de entrenamiento pueden conducir a árboles muy diferentes.

**Bagging** es un procedimiento de propósito general que busca reducir la varianza de una técnica de Machine Learning, pero es particularmente útil para los árboles de decisión. Bagging es la abreviatura de **bootstrap aggregation**, que significa la agregación de muestras bootstrap.

Pero ... ¿Qué es una **muestra bootstrap**? Una muestra aleatoria con reemplazo. Veamos un ejemplo:

In [ ]:
# Definimos una semilla para que nos de los mismos valores
np.random.seed(1)

# Crear un array de 1 x 20
nums = np.arange(1, 21)
print(nums)

# Obtener una muestra aleatoria de tamaño 20 con reemplazo
print(np.random.choice(a=nums, size=20, replace=True))

**¿Cómo funciona el bagging (para árboles de decisión)? **

1. Construir B árboles utilizando B muestras de arranque (bootstrap samples) de los datos de entrenamiento.
2. Entrenar a cada árbol en su muestra de arranque (bootstrap sample) y haga predicciones.
3. Combinar las predicciones:
     - Promedio de las predicciones para ** árboles de regresión **
     - Haz un voto para ** árboles de clasificación **

Notas:

- ** Cada muestra de arranque ** debe ser del mismo tamaño que el conjunto de entrenamiento original.
- ** B ** debe ser un valor suficientemente grande para que el error parezca haberse "estabilizado".
- Los árboles ** crecen en profundidad ** por lo que tienen alta varianza.

Bagging aumenta la precisión predictiva al ** reducir la varianza **, similar a cómo la validación cruzada reduce la varianza asociada con la división entrenamiento / prueba (para estimar el error fuera de muestra) dividiendo muchas veces el promedio de los resultados.

In [ ]:
# Definimos una semilla para que nos de los mismos valores
np.random.seed(123)

n_samples = train.shape[0]
n_B = 10

# Crear 10 muestras bootstrap (seran usandos para elegir los registros del DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples

In [ ]:
# Mostrar las filas para el primer árbol de decisión
train.iloc[samples[0], :]

Construir un árbol de decisión por cada muestra

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Instanciar el árbol de regresión
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)

# Crear un DataFrame para almacenar las predicciones del precio por cada árbol
y_pred = pd.DataFrame(index=test.index, columns=[list(range(n_B))])

# Entrenar un árbol por cada muestra bootstrap y hacer predicciones en el dataset de prueba
for i, sample in enumerate(samples):
    X_train = train.loc[sample, predictores]
    y_train = train.loc[sample, 'price']
    treereg.fit(X_train, y_train)
    y_pred[i] = treereg.predict(X_test)

In [ ]:
y_pred

Resultados de cada Árbol

In [ ]:
for i in range(n_B):
    print(i, np.sqrt(mean_squared_error(y_pred[i], y_test)))

Resultados del Ensamblado

In [ ]:
y_pred.mean(axis=1)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred.mean(axis=1)))

## Árboles de Decisión con Bagging en scikit-learn (with B=500)

In [ ]:
# Definir los conjuntos de entrenamiento y test
X_train = train.loc[:, predictores]
y_train = train.loc[:, 'price']
X_test = test.loc[:, predictores]
y_test = test.loc[:, 'price']

In [ ]:
# Entrenar un BaggingRegressor usando DecisionTreeRegressor como "estimador base"
from sklearn.ensemble import BaggingRegressor
bagreg = BaggingRegressor(DecisionTreeRegressor(), n_estimators=500, 
                          bootstrap=True, oob_score=True, random_state=1)

In [ ]:
# Ajustar y predecir
bagreg.fit(X_train, y_train)
y_pred = bagreg.predict(X_test)
y_pred

In [ ]:
# calculate RMSE
np.sqrt(mean_squared_error(y_test, y_pred))

## Estimando el error out-of-sample

Para los modelos que usan bagging, el error out-of-sample se puede estimar sin usar la **división en entrenamiento/prueba** o la **validación cruzada**

En promedio cada bagged tree usa aproximadamente **dos tercios** (~66%) de las observaciones. Para cada árbol, las **observaciones restantes** se llaman observaciones "out-of-bag".

In [ ]:
# Mostrar la primera muestra bootstrap
samples[0]

In [ ]:
# Mostar las observaciones "en-bolsa" por cada muestra
porcentaje_average=0
for sample in samples:
    print(set(sample))
    print((len(set(sample))*100.0)/n_samples)
    porcentaje_average+=(len(set(sample))*100.0)/n_samples
porcentaje_average/len(samples)

In [ ]:
# Mostrar las observaciones "fuera-de-bolsa" por cada muestra
for sample in samples:
    print(sorted(set(range(n_samples)) - set(sample)))

Cómo calcular el error **"out-of-bag"**:

1. Para cada observación en los datos de entrenamiento, prediga su valor de respuesta usando ** solo ** los árboles en los que esa observación fue un out-of-bag. Promedia esas predicciones (para la regresión) o toma un voto (para la clasificación).
2. Compare todas las predicciones con los valores de respuesta reales para calcular el error out-of-bag error.

Cuando B es suficientemente grande, el  **out-of-bag error**  es una estimación precisa de **out-of-sample error**.

In [ ]:
# Calcular el error "fuera de bolsa" (Coeficiente de Determinacion) para B=500
bagreg.oob_score_

In [ ]:
X_train.loc[~bagreg.estimators_samples_[0]]

In [ ]:
X_train['index'] = X_train.index

In [ ]:
X_train

In [ ]:
dRowPred = {}
for i in range(X_train.shape[0]):
    print("Registro #{0}".format(i))
    dRowPred[i] = []
    for j in range(500):
        if i in X_train.loc[~bagreg.estimators_samples_[j], 'index'].unique().tolist():
            dRowPred[i].append(bagreg.estimators_[j].predict(X_train.loc[[i], predictores])[0])
            #print(bagreg.estimators_[j].predict(X_train.loc[[i], predictores])[0])

In [ ]:
dRowAvgPred = []
for i in dRowPred.keys():
    dRowAvgPred.append({'index':i ,'pred': np.mean(dRowPred[i])})

In [ ]:
from sklearn.metrics import r2_score

El coeficiente determina la calidad del modelo para replicar los resultados.

In [ ]:
r2_score(y_train, pd.DataFrame(dRowAvgPred)['pred'])

In [ ]:
np.sqrt(mean_squared_error(y_train, pd.DataFrame(dRowAvgPred)['pred']))

## Estimar la importancia de las características

Usar Bagging aumenta **la precisión predictiva**, pero disminuye la **interpretabilidad del modelo** porque ya no es posible visualizar el árbol para comprender la importancia de cada característica.

Sin embargo, aún podemos obtener un resumen general de las **importancias de la variables** de los modelos con bagging:

- **Bagged regression trees:** calcula la cantidad total de **MSE** que se reduce debido a divisiones sobre una característica dada, promediada sobre todos los árboles.
- **Bagged classification trees:** calcula la cantidad total del **índice de Gini** que disminuye debido a las divisiones sobre una característica determinada, promediada en todos los árboles.

# 4. Random Forests

Random Forests es una **ligera variación de árboles con bagging** que tiene un rendimiento aún mejor:

- Exactamente como el bagging, creamos un conjunto de árboles de decisión utilizando muestras bootstrap del conjunto de entrenamiento.
- Sin embargo, al construir cada árbol, cada vez que se considera una división, se elige una **muestra aleatoria de m características** como candidatos para dividir del **conjunto completo de características p**. La división solo permite usar **una de esas m características**.
    - Se elige una nueva muestra aleatoria de características para ** cada árbol en cada división individual **.
    - Para ** clasificación **, se elige m típicamente como la raíz cuadrada de p.
    - Para ** regresión **, se elige m típicamente entre p/3 y p.

¿Cuál es el motivo?
- Supongamos que hay ** una característica muy fuerte ** en el conjunto de datos. Cuando se usan árboles con bagging, la mayoría de los árboles usarán esa característica como la división superior, lo que resulta en un conjunto de árboles similares **altamente correlacionados**.
- Promediar cantidades altamente correlacionadas no reduce significativamente la varianza (que es el objetivo total del bagging).
-  Al omitir aleatoriamente las características candidatas de cada división, **Random Forests "decorrelaciona" los árboles**, de modo que el proceso de promediado puede reducir la varianza del modelo resultante.

# 5. Construir y afinar (tuning) Árboles de Decisiones y  Random Forests

- Datos de clientes de un Banco (variables sociodemograficas, de comportamiento)
- Cada observación representa un cliente
- ** Objetivo: ** Predecir la aceptación de un crédito hipotecario

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Lectura del dataset
df = pd.read_csv('data/DS_Credito Hipotecario.csv')

df['SEXO'].fillna(df['SEXO'].mode()[0], inplace=True)
df['FLAG_CASADO'].fillna(df['FLAG_CASADO'].mode()[0], inplace=True)
df['NRO_DEPENDIENTES'].fillna(df['NRO_DEPENDIENTES'].mode()[0], inplace=True)
df['FLAG_TRAB_INDEP'].fillna(df['FLAG_TRAB_INDEP'].mode()[0], inplace=True)
df['INGRESOS_COSOLICITANTE'].fillna(df['INGRESOS_COSOLICITANTE'].mean(), inplace=True)
df['MONTO_PRESTAMO_MILES'].fillna(df['MONTO_PRESTAMO_MILES'].mean(), inplace=True)
df['PLAZO_PRESTAMO_MESES'].fillna(df['PLAZO_PRESTAMO_MESES'].mean(), inplace=True)
df['FLAG_HISTORIAL_CREDITICIO'].fillna(df['FLAG_HISTORIAL_CREDITICIO'].mode()[0], inplace=True)

df['SEXO'] = df['SEXO'].map({'Male': 0, 'Female': 1})
df['FLAG_CASADO'] = df['FLAG_CASADO'].map({'No': 0, 'Yes': 1})
df['EDUCACION'] = df['EDUCACION'].map({'Not Graduate': 0, 'Graduate': 1})
df['FLAG_TRAB_INDEP'] = df['FLAG_TRAB_INDEP'].map({'No': 0, 'Yes': 1})
df['FLAG_HISTORIAL_CREDITICIO'] = df['FLAG_HISTORIAL_CREDITICIO'].map({'Malo': 0, 'Bueno': 1})
df = pd.concat([df, pd.get_dummies(df['TIPO_ZONA'], prefix = 'TIPO_ZONA', drop_first = True)], axis=1)
df = pd.concat([df, pd.get_dummies(df['NRO_DEPENDIENTES'], prefix = 'NRO_DEPENDIENTES', drop_first = True)], axis=1)
del df['TIPO_ZONA']
del df['NRO_DEPENDIENTES']
df['FLAG_CRED_HIPO'] = df['FLAG_CRED_HIPO'].map({'Y': 1, 'N': 0})

In [ ]:
# Definir X e Y
predictores = ['SEXO', 'FLAG_CASADO', 'EDUCACION', 'FLAG_TRAB_INDEP', 'INGRESOS_SOLICITANTE', 'INGRESOS_COSOLICITANTE',
                'MONTO_PRESTAMO_MILES', 'PLAZO_PRESTAMO_MESES','FLAG_HISTORIAL_CREDITICIO', 'TIPO_ZONA_Semiurban',
                'TIPO_ZONA_Urban', 'NRO_DEPENDIENTES_1', 'NRO_DEPENDIENTES_2','NRO_DEPENDIENTES_3+']
X = df[predictores]
y = df.FLAG_CRED_HIPO

## Predecir el Crédito Hipotecario con un árbol de decisión

Encuentre el mejor ** max_depth ** para un árbol de decisión usando la validación cruzada:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Lista de valores para "max_depth"
max_depth_range = range(1, 21)

# Definir un lista para almacenar los promedios de los Auc por cada valor de "max_depth"
auc_scores = []

# Usa una validación cruzada de 10 folds con cada valor del "max_depth"
from sklearn.cross_validation import cross_val_score
for depth in max_depth_range:
    treecla = DecisionTreeClassifier(max_depth=depth, random_state=1)
    auc_score = cross_val_score(treecla, X, y, cv=10, scoring='roc_auc')
    auc_scores.append(np.mean(auc_score))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')

In [ ]:
# Graficar max_depth (eje x) versus AUC (eje y)
plt.plot(max_depth_range, auc_scores)
plt.xlabel('max_depth')
plt.ylabel('AUC')

In [ ]:
# Mostra el mejor AUC y el correspondiente "max_depth"
sorted(zip(auc_scores, max_depth_range), reverse=True)[0]

In [ ]:
# max_depth=3 fue el mejor, por lo que hay que entrenar un arbol usando ese valor de hiperparametro
treecla = DecisionTreeClassifier(max_depth=3, random_state=1)
treecla.fit(X, y)

In [ ]:
# Calcular la importancia de variables
pd.DataFrame({'predictor':predictores, 'importancia':treecla.feature_importances_}).sort_values('importancia', ascending=False)

## Predecir el Crédito Hipotecario con Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
rfcla = RandomForestClassifier()
rfcla

### Tuning n_estimators

Un hiperparámetro importante es **n_estimators**, que es la cantidad de árboles que se deben entrenar. Debe ser un valor lo suficientemente grande como para que el error parezca haberse "estabilizado".

In [ ]:
# Listar los valores a probar
estimator_range = range(10, 310, 10)

# Definir un lista para almacenar los promedios de los Auc por cada valor de "max_depth"
AUC_scores = []

# Usa una validación cruzada de 5 folds con cada valor del "n_estimators"
for estimator in estimator_range:
    rfcla = RandomForestClassifier(n_estimators=estimator, random_state=1, n_jobs=-1)
    AUC_score = cross_val_score(rfcla, X, y, cv=5, scoring='roc_auc')
    AUC_scores.append(np.mean(AUC_score))

In [ ]:
# Graficar n_estimators (eje x) versus AUC (eje y)
plt.plot(estimator_range, AUC_scores)
plt.xlabel('n_estimators')
plt.ylabel('AUC')

In [ ]:
sorted(zip(AUC_scores, estimator_range), reverse=True)[0]

### Tuning max_features

El otro parámetro de ajuste importante es ** max_features **, que es el número de características que se deben considerar en cada división.

In [ ]:
# Listar los valores para "max_features"
feature_range = range(1, len(predictores)+1)

# Definir un lista para almacenar los promedios de los Auc por cada valor de "max_features"
AUC_scores = []

# Use una validación cruzada de 10 folds para cada valor de "max_features"
for feature in feature_range:
    rfdec = RandomForestClassifier(n_estimators=150, max_features=feature, random_state=1, n_jobs=-1)
    AUC_score = cross_val_score(rfdec, X, y, cv=10, scoring='roc_auc')
    AUC_scores.append(np.mean(AUC_score))

In [ ]:
# Graficar max_features (eje x) versus AUC (eje y)
plt.plot(feature_range, AUC_scores)
plt.xlabel('max_features')
plt.ylabel('AUC')

In [ ]:
# Mostrar el mejor AUC y su correpondiente "max_features"
sorted(zip(AUC_scores, feature_range), reverse=True)[0]

### Ajustando un Random Forest con los mejores parámetros

In [ ]:
# max_features=6 is best and n_estimators=150 is sufficiently large
rfreg = RandomForestRegressor(n_estimators=60, max_features=6, oob_score=True, random_state=1)
rfreg.fit(X, y)

In [ ]:
rfreg.predict(X[5:100])

In [ ]:
rfreg

In [ ]:
# compute feature importances
pd.DataFrame({'feature':predictores, 'importance':rfreg.feature_importances_}).sort_values('importance')

In [ ]:
# compute the out-of-bag R-squared score
rfreg.oob_score_

### Reduciendo X a sus características más importantes


In [ ]:
# Verificar las dimensiones de X
X.shape

In [ ]:
# Establecer un umbral para qué características incluir
print(rfreg.transform(X, threshold=0.1).shape)
print(rfreg.transform(X, threshold='mean').shape)
print(rfreg.transform(X, threshold='median').shape)

In [ ]:
# create a new feature matrix that only includes important features
X_important = rfreg.transform(X, threshold='mean')

In [ ]:
# check the RMSE for a Random Forest that only includes important features
rfreg = RandomForestRegressor(n_estimators=150, max_features=3, random_state=1)
scores = cross_val_score(rfreg, X_important, y, cv=10, scoring='mean_squared_error')
np.mean(np.sqrt(-scores))

## Comparando Random Forests con Árboles de Decisiones

**Ventajas de Random Forests:**

- El rendimiento es competitivo con los mejores métodos de aprendizaje supervisado
- Proporciona una estimación más confiable de la importancia de las características (features)
- Le permite estimar el error fuera de muestra sin usar entrenamiento / prueba dividida o validación cruzada

**Desventajas de Random Forests:**

- Menos interpretable
- Más lento para entrenar
- Más lento para predecir

![Machine learning flowchart](https://raw.githubusercontent.com/justmarkham/DAT8/master/notebooks/images/driver_ensembling.png)

*Machine learning flowchart created by the [second place finisher](http://blog.kaggle.com/2015/04/20/axa-winners-interview-learning-telematic-fingerprints-from-gps-data/) of Kaggle's [Driver Telematics competition](https://www.kaggle.com/c/axa-driver-telematics-analysis)*